In [1]:
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 将数据导入
mnist  = input_data.read_data_sets('./data', one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("MNIST Ready")

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
MNIST Ready


In [3]:
# 参数初始化操作
n_input = 784
n_output = 10
weights = {
    'wc1':tf.Variable(tf.random_normal([3,3,1,64], stddev=0.1)), # 第一个卷积层  因为原始的是灰度图 所以第三维为1
    'wc2':tf.Variable(tf.random_normal([3,3,64,128], stddev=0.1)), # 第二个卷积层
    'wd1':tf.Variable(tf.random_normal([7*7*128,1024], stddev=0.1)), # 第一个全连接层
    'wd2':tf.Variable(tf.random_normal([1024, n_output], stddev=0.1)), # 第二个全连接层
}

biases = {
    'bc1':tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2':tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1':tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'bd2':tf.Variable(tf.random_normal([n_output], stddev=0.1)),
}

In [4]:
# 卷积+池化
def conv_basic(_input, _w, _b, _keepratio): # keepratio drop层每次训练保留的比率
    # 输入
    _input_r = tf.reshape(_input, shape=[-1,28,28,1])   # 输入数据的格式： batch_size*high*weight*depth
    # 第一层卷积操作
    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1,1,1,1], padding='SAME') # stride 步幅
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1,_b['bc1'])) # relu激活函数
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # 使用max进行池化操作
    _pool_dr1 = tf.nn.dropout(_pool1,_keepratio) # dropout层
    
    
    # 第二层卷积操作
    _conv2 = tf.nn.conv2d(_pool_dr1, _w['wc2'], strides=[1,1,1,1], padding='SAME') # stride 步幅
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2,_b['bc2'])) # relu激活函数
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # 使用max进行池化操作
    _pool_dr2 = tf.nn.dropout(_pool2,_keepratio) # dropout层
    
    # 将多维转化为1维
    _dense1 = tf.reshape(_pool_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
    
    # 第一个全连接层
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    
    # 第一个全连接层
    _out = tf.add(tf.matmul(_fc_dr1,_w['wd2']), _b['bd2'])
    
    # return
    out = {
        'input_r':_input_r, 'conv1':_conv1, 'pool1':_pool1,'pool_dr1':_pool_dr1,
        'conv2':_conv2, 'pool2':_pool2,'pool_dr2':_pool_dr2,'dense1':_dense1,
        'fc1':_fc1,'fc_dr1':_fc_dr1,'out':_out
    }
    
    return out
print("CNN Ready")

CNN Ready


In [5]:
x = tf.placeholder(tf.float32,[None, n_input])
y = tf.placeholder(tf.float32,[None, n_output])
keepratio = tf.placeholder(tf.float32)

# tensorflow一些操作
_pred = conv_basic(x, weights, biases, keepratio)['out'] # 预测函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y)) # 计算损失函数
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) # 优化器

_corr = tf.equal(tf.argmax(_pred, 1), tf.argmax(y, 1))  
accr = tf.reduce_mean(tf.cast(_corr,tf.float32)) # 计算准确度

init = tf.global_variables_initializer() # 初始化操作

# 模型的保存相关设置
# SAVER
save_step = 1 # 每epoch就保存一次
saver = tf.train.Saver(max_to_keep=3)   # 保存的参数的最大范围  设置为保存3次迭代的参数

print("GRAPH READY")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

GRAPH READY


In [6]:
do_train = 1
sess = tf.Session()
sess.run(init)

In [7]:
training_epochs = 15
batch_size = 16
display_step = 1
if do_train == 1:
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = 10
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optm, feed_dict={x:batch_xs,y:batch_ys, keepratio:0.7 })
            # 计算平均误差
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys, keepratio:1.})/total_batch

        if epoch % display_step == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc =sess.run(accr, feed_dict={x:batch_xs, y:batch_ys, keepratio:1. })
            print("Traning accuracy: %.9f" % (train_acc))

#             test_acc = sess.run(accr, feed_dict={x:testimg, y:testlabel, keepratio:1. })
#             print("Test accuracy: %.3f" % (test_acc))
        
        # 进行模型的保存操作
        if epoch % save_step == 0:
            saver.save(sess, "save/nets/cnn_mnist_basic.ckpt-" + str(epoch))
    print("OPTIMIZATION FINISHED")

Epoch: 000/015 cost: 5.622116637
Traning accuracy: 0.187500000
Epoch: 001/015 cost: 2.898527050
Traning accuracy: 0.375000000
Epoch: 002/015 cost: 1.442147481
Traning accuracy: 0.562500000
Epoch: 003/015 cost: 1.255333036
Traning accuracy: 0.500000000
Epoch: 004/015 cost: 1.204416513
Traning accuracy: 0.750000000
Epoch: 005/015 cost: 1.177002358
Traning accuracy: 0.937500000
Epoch: 006/015 cost: 0.976815331
Traning accuracy: 0.562500000
Epoch: 007/015 cost: 0.992108470
Traning accuracy: 0.687500000
Epoch: 008/015 cost: 0.922193468
Traning accuracy: 0.750000000
Epoch: 009/015 cost: 0.803165436
Traning accuracy: 0.875000000
Epoch: 010/015 cost: 0.839902925
Traning accuracy: 0.750000000
Epoch: 011/015 cost: 0.714129961
Traning accuracy: 0.625000000
Epoch: 012/015 cost: 0.617662951
Traning accuracy: 0.750000000
Epoch: 013/015 cost: 0.654907161
Traning accuracy: 0.750000000
Epoch: 014/015 cost: 0.611684394
Traning accuracy: 1.000000000
OPTIMIZATION FINISHED


In [8]:
do_train = 0
sess = tf.Session()
sess.run(init)

In [9]:
if do_train == 0:
    epoch = training_epochs-1
    saver.restore(sess, "save/nets/cnn_mnist_basic.ckpt-" + str(epoch))
    
    test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel, keepratio:1.})
    print (" TEST ACCURACY: %.3f" % (test_acc))

INFO:tensorflow:Restoring parameters from save/nets/cnn_mnist_basic.ckpt-14
 TEST ACCURACY: 0.865
